In [9]:
from dotenv import load_dotenv
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient
from langchain_community.utilities import SQLDatabase
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage, ToolMessage, SystemMessage
from langgraph.prebuilt import  ToolNode, tools_condition
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import List, Annotated
from typing_extensions import TypedDict
from langgraph.runtime import Runtime
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [8]:
class EmailState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    authenticated: bool

class EmailContext(TypedDict):
    email_address: str
    password: str

In [7]:
@tool
def check_inbox() -> str:
    """Check the inbox for recent emails"""
    return """
    Hi Julie, 
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, Jane (jane@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send a response email"""
    return f"Email sent to {to} with subject '{subject}' and body:\n\n{body}"

@tool
def authenticate(email: str, password: str) -> str:
    """Authenticate the user. On success, unlocks email tools."""
    # In real app: validate against DB or context
    # Here we simulate correct credentials
    if email == "julie@example.com" and password == "secret123":
        return "Authentication successful! You can now check inbox and send emails."
    else:
        return "Authentication failed. Wrong email or password."

In [10]:
def chatNode(state: EmailState, runtime: Runtime[EmailContext]) -> dict:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    # Dynamic system prompt
    if state.get("authenticated", False):
        system_prompt = "You are a helpful email assistant. You can check the inbox and send emails."
    else:
        system_prompt = "You are a helpful assistant. First, ask the user to authenticate with email and password."
    
    messages = [SystemMessage(content=system_prompt)] + state["messages"]

    # Dynamic tools based on auth status
    if state.get("authenticated", False):
        tools = [check_inbox, send_email]
    else:
        tools = [authenticate]
    
    llm_with_tools = llm.bind_tools(tools)

    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}